# FIRST_VALUE

Cours : [DLC] SQL pour Débutant(e)s Jour 7 - Synthèse et Techniques Avancées
Lien : https://www.youtube.com/watch?v=9Aj9X-j1Yys&list=PLyh35eYRez8cAEBfi1LWN70Cs77bBvCOp&index=7

Github le coin stat : https://github.com/LeCoinStat/LeCoinStat/tree/main/Bien_Debuter_SQL
Fichiers Github récupérés : https://github.com/LeCoinStat/LeCoinStat/tree/main/Bien_Debuter_SQL/Jour7

> Syntaxe :
> ```
> FIRST_VALUE(expression) OVER(expression)
> ```
> <br>

In [1]:
import duckdb
from skimpy import skim

In [2]:
# Connection à la librairie duckdb
conn = duckdb.connect()

## Fichier des ventes

In [3]:
# Requête SQL : récupération de toutes les données du fichier .csv en DF
sales_df = conn.execute(
    """
    SELECT *
    FROM read_csv_auto('data/indexE/Ventes.csv', header=True)
    """
).df()

In [4]:
# Stats de la DF
skim(sales_df)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 100    │ │ int32       │ 5     │                                                          │
│ │ Number of columns │ 7      │ │ datetime64  │ 1     │                                                          │
│ └───────────────────┴────────┘ │ float64     │ 1     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┓  │
│ ┃ column_name        ┃ NA  ┃ NA %   ┃ mean    ┃ sd     ┃ p0     ┃ p25    ┃ p50   ┃ p75    ┃ p100   ┃ hist    ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━┩  │
│ │ VenteID            │   0 │      0 │    50.5 │  29.01 │      1 │  25.75 │  50.5 │  75.25 │    100 │ ▇▇▇▇▇▇  │  │
│ │ ClientID           │   0 │      0 │   51.26 │  29.18 │      1 │  27.75 │  53.5 │  77.25 │     99 │ ▆▇▅▇▆▇  │  │
│ │ EmployeID          │   0 │      0 │   50.65 │   29.4 │      2 │     23 │  49.5 │  76.25 │    100 │ ▇▇▇▆▇▇  │  │
│ │ ProduitID          │   0 │      0 │   48.08 │  24.56 │      3 │     29 │    46 │  65.25 │     98 │ ▅▆▇▇▅▃  │  │
│ │ QuantiteVendue     │   0 │      0 │   48.36 │  30.17 │      1 │     22 │  50.5 │     71 │    100 │ ▇▇▃▇▃▇  │  │
│ │ MontantTotal       │   0 │      0 │    8278 │  11580 │  59.98 │   1542 │  2979 │   9249 │  48500 │ ▇▁▁  ▁  │  │
│ └────────────────────┴─────┴────────┴─────────┴────────┴────────┴────────┴───────┴────────┴────────┴─────────┘  │
│                                                    datetime                                                     │
│ ┏━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓  │
│ ┃ column_name      ┃ NA   ┃ NA %    ┃ first                      ┃ last                       ┃ frequency    ┃  │
│ ┡━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩  │
│ │ DateVente        │    0 │       0 │    2020-01-08 12:43:33     │    2023-12-27 15:39:42     │ None         │  │
│ └──────────────────┴──────┴─────────┴────────────────────────────┴────────────────────────────┴──────────────┘  │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯

In [5]:
# Enregistrement de la DF en BD virtuelle (requête SQL : VIEW)
conn.register('sales_db', sales_df)

## Requêtes SQL

Pour chaque client, quel était le premier produit qu'il a acheté

In [13]:
firstProduct_df = conn.execute(
    """
    -- Récupérationd des colonnes
    SELECT
        s.ClientID
        , s.DateVente AS "Date d'achat"
        , FIRST_VALUE(s.ProduitID) OVER(
            -- Création autant de table que d'ID clients
            PARTITION BY ClientID
            -- Trie par date
            ORDER BY DateVente
            -- Récupération de toutes les lignes
            ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
            ) AS "N° ID du premier achat"
    -- Récupération des BD
    FROM sales_db s
    """
).df()

In [14]:
# Conversion du champ date en str
firstProduct_df["Date d'achat"] = firstProduct_df[
    "Date d'achat"].dt.strftime('%d/%m/%Y')

In [15]:
firstProduct_df

,ClientID,Date d'achat,N° ID du premier achat
0,11,18/12/2020,41
1,55,17/02/2023,50
2,55,24/06/2023,50
3,83,29/02/2020,24
4,83,19/12/2020,24
...,...,...,...
95,33,22/02/2022,61
96,33,19/11/2023,61
97,40,16/10/2023,57
98,51,15/12/2020,37
